In [ ]:
# ## Import Libraries

# Standard Libraries
import warnings

# Scientific Libraries
import numpy as np
import optuna
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error, recall_score

# Sklearn
from sklearn.preprocessing import StandardScaler

# Torch Tools
from torch.utils.tensorboard import SummaryWriter

# Custom Utilities
from utils.dl_helper_functions import (
    convert_to_tensors,
    create_sequences,
    load_picture_lagged_data,
    scale_data,
)
from utils.Model_ConvLSTM import CNNLSTMWaterLevelModel
from utils.Model_Training import training_ConvLSTM

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Suppress specific warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# Display all columns in pandas
pd.options.display.max_columns = None

# TensorBoard writer
writer = SummaryWriter()

# Define global constants
DTYPE_NUMPY = np.float32              # Datentyp für numpy Arrays
n_jobs = -1                           # Anzahl CPUs für parallele Prozesse
HORIZON = 24 * 3                      # 3 Tage Vorhersagehorizont
INITIAL_TRAINING_SIZE = 24 * 183     # 6 Monate Trainingsdaten (4392 Stunden)
SEQUENCE_LENGTH = 24                 # 1 Tag als Input-Sequenz

# Load data
X, y_lagged, y, common_time = load_picture_lagged_data(
    return_common_time=True,
    verbose=False,
    grid_size=25,
    n_jobs=n_jobs,
    dtype=DTYPE_NUMPY,
    pca=False
)

# Daten vorbereiten
X = X.astype(DTYPE_NUMPY)
y_lagged = y_lagged.astype(DTYPE_NUMPY)
y = y.astype(DTYPE_NUMPY)

# Cross-Validation Zeitpunkte
folds = {
    "Surge1": pd.Timestamp("2023-02-25 16:00:00"),
    "Surge2": pd.Timestamp("2023-04-01 09:00:00"),
    "Surge3": pd.Timestamp("2023-10-07 20:00:00"),
    "Surge4": pd.Timestamp("2023-10-20 21:00:00"),
    "Surge5": pd.Timestamp("2024-01-03 01:00:00"),
    "Surge6": pd.Timestamp("2024-02-09 15:00:00"),
    "Surge7": pd.Timestamp("2024-12-09 10:00:00"),
    "normal1": pd.Timestamp("2023-07-01 14:00:00"),
    "normal2": pd.Timestamp("2024-04-01 18:00:00"),
    "normal3": pd.Timestamp("2025-01-01 12:00:00"),
}


/gxfs_home/geomar/smomw693/Documents/GEOMAR-DeepLearning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 20161/20161 [00:02<00:00, 7331.48it/s]


In [ ]:


def custom_score(y_true=None, y_pred=None, bins=[1, 2.00], alpha=0.7):
    
    # Initialisiere Recall- und MSE-Werte
    recalls = []
    for i in range(y_true.shape[1]):  # Iteriere über jede Spalte
        y_true_class = np.digitize(y_true[:, i], bins=bins)
        y_pred_class = np.digitize(y_pred[:, i], bins=bins)
        recalls.append(recall_score(y_true_class, y_pred_class, average="macro"))
    
    mean_recall = np.mean(recalls)  # Durchschnittlicher Recall
    mse = mean_squared_error(y_true, y_pred)
    return alpha * (1 - mean_recall) + (1 - alpha) * mse

def cross_validation_loop(model_name, folds, X, y_lagged, y, common_time, time_delta, trial_params):
    fold_results = []

    for surge_name, fold in folds.items():
        start_cutoff = fold - time_delta
        end_cutoff = fold + time_delta
        idx_start_cutoff = np.where(common_time == start_cutoff)[0][0]
        idx_end_cutoff = np.where(common_time == end_cutoff)[0][0]

        X_test = X[idx_start_cutoff:idx_end_cutoff]
        y_lagged_test = y_lagged[idx_start_cutoff:idx_end_cutoff]
        y_test = y[idx_start_cutoff:idx_end_cutoff]

        X_train = X.copy()
        y_lagged_train = y_lagged.copy()
        y_train = y.copy()
        X_train[idx_start_cutoff:idx_end_cutoff] = np.nan
        y_lagged_train[idx_start_cutoff:idx_end_cutoff] = np.nan
        y_train[idx_start_cutoff:idx_end_cutoff] = np.nan

        X_train, y_lagged_train, y_train = create_sequences(X_train, y_lagged_train, y_train, SEQUENCE_LENGTH, HORIZON)
        X_test, y_lagged_test, y_test = create_sequences(X_test, y_lagged_test, y_test, SEQUENCE_LENGTH, HORIZON)

        gap = 168
        X_test = X_test[gap:-gap]
        y_lagged_test = y_lagged_test[gap:-gap]
        y_test = y_test[gap:-gap]

        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_train, y_lagged_train, y_train, _, _, _, X_test, y_lagged_test, y_test = scale_data(
            X_scaler=scaler_X, y_lagged_scaler=scaler_y,
            X_train=X_train, y_lagged_train=y_lagged_train, y_train=y_train,
            X_val=None, y_lagged_val=None, y_val=None,
            X_test=X_test, y_lagged_test=y_lagged_test, y_test=y_test,
            dtype=DTYPE_NUMPY, verbose=False
        )

        X_train_tensor, y_lagged_train_tensor, y_train_tensor, _, _, _, X_test_tensor, y_lagged_test_tensor, y_test_tensor = convert_to_tensors(
            X_train=X_train, y_lagged_train=y_lagged_train, y_train=y_train,
            X_val=None, y_lagged_val=None, y_val=None,
            X_test=X_test, y_lagged_test=y_lagged_test, y_test=y_test,
            dtype=torch.float32
        )

        model = CNNLSTMWaterLevelModel(
            in_channels=X_train_tensor.shape[2],
            forecast_horizon=HORIZON,
            lagged_input_dim=y_lagged_train_tensor.shape[2],
            H=X_train_tensor.shape[3],
            W=X_train_tensor.shape[4],
            cnn1_out_channels=trial_params["cnn1_out_channels"],
            cnn2_out_channels=trial_params["cnn2_out_channels"],
            cnn1_kernel_size=3,
            cnn1_padding=1,
            cnn2_kernel_size=3,
            cnn2_padding=1,
            cnn_linear_out_features=trial_params["cnn_linear_out_features"],
            lstm_hidden_dim=trial_params["lstm_hidden_dim"],
            lstm_layers=trial_params["lstm_layers"],
            lstm_input_size=trial_params["cnn_linear_out_features"],
            dropout=trial_params["dropout"]
        )

        optimizer = torch.optim.Adam(model.parameters(), lr=trial_params["lr"])

        best_model = training_ConvLSTM(
            model,
            X_train=X_train_tensor,
            y_train=y_train_tensor,
            X_val=X_test_tensor,
            y_val=y_test_tensor,
            y_lagged_train=y_lagged_train_tensor,
            y_lagged_val=y_lagged_test_tensor,
            epochs=trial_params["epochs"],
            batch_size=128,
            optimizer=optimizer,
            writer=None,
            verbose=True,
            log_tensorboard=False
        )

        model.eval()
        with torch.no_grad():
            y_pred = model.predict(X_test_tensor, y_lagged_test_tensor).cpu().numpy()
            y_true = y_test_tensor.cpu().numpy()

        score = custom_score(y_true=y_true, y_pred=y_pred, bins=[1, 2.00], alpha=0.7)
        fold_results.append(score)

    return fold_results


def objective(trial):
    trial_params = {
        "cnn1_out_channels": trial.suggest_categorical("cnn1_out_channels", [16, 32, 64]),
        "cnn2_out_channels": trial.suggest_categorical("cnn2_out_channels", [32, 64, 128]),
        "cnn_linear_out_features": trial.suggest_categorical("cnn_linear_out_features", [64, 128, 256]),
        "lstm_hidden_dim": trial.suggest_categorical("lstm_hidden_dim", [32, 64, 128]),
        "lstm_layers": trial.suggest_int("lstm_layers", 1, 3),
        "dropout": trial.suggest_float("dropout", 0.1, 0.6),
        "lr": trial.suggest_float("lr", 0.0001, 0.1, log=True),
        "epochs": 10,
    }

    scores = cross_validation_loop(
        model_name="ConvLSTM",
        folds=folds,
        X=X.astype(DTYPE_NUMPY),
        y_lagged=y_lagged.astype(DTYPE_NUMPY),
        y=y.astype(DTYPE_NUMPY),
        common_time=common_time,
        time_delta=pd.Timedelta(hours=168 * 4),
        trial_params=trial_params
    )

    return np.mean(scores)


study = optuna.create_study(direction="minimize", study_name="CNNLSTM_Wasserpegel")
study.optimize(objective, n_trials=30)  # z. B. 30 Versuche oder 1 Stunde max

print("Beste Parameter:", study.best_params)
print("Bester Score:", study.best_value)


[I 2025-07-05 21:36:08,746] A new study created in memory with name: CNNLSTM_Wasserpegel


| Epoch 1 | Train Loss: 0.0782, Validation Loss: 0.1117 | Train MSE: 0.0687, Val MSE: 0.1117 |
| Epoch 2 | Train Loss: 0.0686, Validation Loss: 0.1105 | Train MSE: 0.0686, Val MSE: 0.1105 |
| Epoch 3 | Train Loss: 0.0687, Validation Loss: 0.1087 | Train MSE: 0.0873, Val MSE: 0.1087 |
| Epoch 4 | Train Loss: 0.0698, Validation Loss: 0.1089 | Train MSE: 0.0687, Val MSE: 0.1089 |
| Epoch 5 | Train Loss: 0.0687, Validation Loss: 0.1091 | Train MSE: 0.0686, Val MSE: 0.1091 |
